<a href="https://colab.research.google.com/github/rendalamili/ml-for-table-extraction/blob/main/Debug_Table_Transformer_Machine_Learning_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Resources**


https://github.com/ultralytics/yolov5/issues/9679

https://github.com/pytorch/pytorch

https://matplotlib.org/stable/api/pyplot_summary.html

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

https://github.com/Changgun-Choi/Thesis-ViT-Robustness

https://github.com/ultralytics/ultralytics/issues/9403

https://github.com/madmaze/pytesseract

https://huggingface.co/transformers/v4.3.3/_modules/transformers/training_args.html#:~:text=[docs]%20@dataclass%20class%20TrainingArguments:%20%22%22%22%20TrainingArguments%20is%20the%20subset%20of

https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Table%20Transformer/Using_Table_Transformer_for_table_detection_and_table_structure_recognition.ipynb

https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html?highlight=cuda#cuda-tensors

https://github.com/microsoft/table-transformer/blob/main/docs/INFERENCE.md

https://camelot-py.readthedocs.io/en/master/

https://github.com/tesseract-ocr/tesseract

https://www.kaggle.com/datasets/sreesankar711/pubtables-subset-100k

https://github.com/microsoft/table-transformer


In [1]:
# Install packages
!pip install camelot-py[cv] ghostscript PyPDF2==1.26.0 pdf2image transformers matplotlib numpy opencv-python pandas torch tensorflow table-transformer transformers datasets
!apt-get install -y ghostscript
!pip install --upgrade camelot-py[cv]
!pip install pytesseract
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
!pip install timm
!pip install --upgrade transformers
!kaggle datasets download -d sreesankar711/pubtables-subset-100k
!unzip pubtables-subset-100k.zip -d /content/pubtables
!pip install -U 'chitra[all]'

!pip freeze

Streaming output truncated to the last 5000 lines.
  inflating: /content/pubtables/subset/words_val/PMC3009288_table_1_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009288_table_2_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009290_table_0_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009290_table_1_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009290_table_2_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009290_table_3_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009294_table_0_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009294_table_1_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009294_table_2_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009294_table_3_words.json  
  inflating: /content/pubtables/subset/words_val/PMC3009296_table_0_words.json  
  inflating: /content/pubtables/subset/words_val/PMC300929

In [2]:
# Import libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PDF libaries
import camelot
import ghostscript
import PyPDF2 as pypdf
from pdf2image import convert_from_path

# Image processsing libaries
import pytesseract
import cv2
import PIL
from PIL import Image
from PIL import Image, ImageOps, UnidentifiedImageError
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.transforms as transforms
from PyPDF2 import PdfFileReader
import matplotlib.pyplot as plt

# Pytorch libraries
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Transformer and datasets
import transformers
from transformers import TableTransformerForObjectDetection, Trainer, TrainingArguments, pipeline
from transformers import AutoImageProcessor, TableTransformerForObjectDetection, TrainingArguments, Trainer
from transformers import DefaultDataCollator
from transformers import Trainer, TrainingArguments, default_data_collator
from transformers import TapasForQuestionAnswering, pipeline
import datasets
from datasets import load_dataset
from datasets import Dataset, Features, ClassLabel, Value, Image as ImageType

# Chitra
import chitra
from chitra.dataloader import Clf

# Google Collab
from google.colab import files

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [3]:
# Upload PDF file
upload = files.upload()
pdf_file = list(upload.keys())[0]



Saving Table 5_ Table with 1 x Empty Cell.pdf to Table 5_ Table with 1 x Empty Cell.pdf


In [4]:
def extract_pdf_table(pdf_file):
    # Initialise tables list to store extracted tables
    tables = []

    # Extract tables using Camelot (stream)
    try:
        tables = camelot.read_pdf(pdf_file, pages='all', flavor='stream', split_text=True, flag_size=True, strip_text=' .\n')
    except Exception as e:
        print(f"Error extracting table with Camelot (stream): [{e}]")
        # Extract using Camelot (lattice)
        try:
            tables = camelot.read_pdf(pdf_file, pages='all', flavor='lattice', process_background=True, shift_text=[''], line_scale=40, copy_text=['v'], layout_kwargs={'detect_vertical': False})
        except Exception as e:
            print(f"Error extracting table with Camelot (lattice): [{e}]")

    # If tables were sucessfully extracted using Camelot process table and return results
    if tables:
        print(f"Extracted {len(tables)} tables using Camelot.")
        process_table = []
        for table in tables:
            df = table.df # Convert table to Dataframe
            df.columns = df.iloc[0]  # Set first row as column names
            table.df = df  # Update DataFrame in table object
            process_table.append(table.df.values.tolist())  # Convert DataFrame to list of list
        # Return extracted table as list
        return [table.df.values.tolist() for table in tables]

    # If Camelot was unsucessful use OCR
    print("Using OCR for image-based table extraction.")

    # Convert PDF to image using pdf2image
    images = convert_from_path(pdf_file, poppler_path="/usr/bin", fmt='png')

    # Initialise list to store extracted text from images
    extract_text_table = []

    # Extract text from images using Tesseract
    for i, image in enumerate(images):
      image = image = image.convert('RGB')
      text = pytesseract.image_to_string(image)
      extract_text_table.append(text)
      print(f"Extracted text from image [{i + 1}/{len(images)}]")

    # Process extracted text into structured data
    extracted_data = []
    for text in extract_text_table:
        rows = text.split('\n')  # Split text into rows
        for row in rows:
            if row.strip():  # Ignore empty rows
                extracted_data.append(row.strip().split())  # Split into columns

    return extracted_data  # Return structured data

    #upload = files.upload()
    #pdf_file = list(upload.keys())[0]

# Example use of extract_pdf_table function
if __name__ == "__main__":
    data = extract_pdf_table(pdf_file)
    print(data)

Extracted 1 tables using Camelot.
[[['Name', 'Age', 'Country'], ['John', '30', 'USA'], ['Jane', '', 'Canada']]]


In [5]:
# Convert table to JSON
def json_table(table):
  # Check if table has DataFrame 'df' attribute
  if hasattr(table, 'df'):
    header = table.df.values.tolist() # Get df header
    rows = table.df.values.tolist() # Get df rows
    # Return list of dictionaries
    return [dict(zip(header, row)) for row in rows]
  else:
    # If table is a list or tuple
    if table and isinstance(table, (list, tuple)) and len(table) > 1:
      header = table[0] # First element is header
      rows = table[1:] # Remaining elements are rows
      # Return list of dictionaries
      return [dict(zip(header, row)) for row in rows]
    else:
      print("Invalid table format for JSON conversion.")
      return []

In [6]:
# Extract table from PDF file and store as tables
tables = extract_pdf_table(pdf_file)

# Convert extracted table to JSON
json_tables = [json_table(table) for table in tables]
with open('tables.json', 'w') as f:
  json.dump(json_tables, f)
  print(f"Extracted {len(tables)} tables.")

Extracted 1 tables using Camelot.
Extracted 1 tables.


In [7]:
# Convert extracted table to an image
for i, table in enumerate(tables):
  df = pd.DataFrame(table)

  # Create new figure and axis for plotting
  fig, ax = plt.subplots()
  ax.axis('off')

  # Create table plot with DataFrame values and column labels
  plot_table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')

  # Save plot as PNG image
  plt.savefig(f'table_{i}.png', bbox_inches='tight', pad_inches=0)
  plt.close()

In [8]:
# Load and process table image for use in the TableTransformer pipeline
table_data = []
for idx, table in enumerate(json_tables):
  image_path = f'table_{idx}.png'
  table_content = table
  table_data.append((image_path, table_content))

# Pytorch class to create a dataset
class TableData(torch.utils.data.Dataset):
  # Initilises TableData class
  def __init__(self, data):
    self.data = data # Dataset
    self.transform = transforms # Preprocessing

  # Returns length of dataset
  def __len__(self):
    return len(self.data)

  # Access dataset in index
  def __getitem__(self, idx):
    image_path, table_content = self.data[idx] # Gets data from index
    image = Image.open(image_path).convert("RGB") # Open image using PIL as RGB
    inputs = self.processor(images=image, return_tensors="pt") # Preprocess as tensor

    # Dictionary to store tensors
    target = {
        'boxes': torch.tensor([]),
        'labels': torch.tensor([]),
        'image_id': torch.tensor([])
    }

    # Returns dictionary of processed input data and target data
    return {**inputs, **target}

In [9]:
# Load pre-trained Table Transformer model
model = TableTransformerForObjectDetection.from_pretrained('microsoft/table-transformer-detection')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Load image processor
image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")

preprocessor_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [11]:
def load_dataset_with_labels(images_dir, annotations_dir, limit=100):
    dataset = {}

    for filename in os.listdir(annotations_dir):
        if filename.endswith('_words.json'):
            with open(os.path.join(annotations_dir, filename)) as f:
                annotations = json.load(f)

            image_name = filename.replace('_words.json', '.jpg')
            image_path = os.path.join(images_dir, image_name)

            if os.path.exists(image_path):
                if image_path not in dataset:
                    dataset[image_path] = []

                # Collect annotations
                for item in annotations:
                    labels = {
                        'text': item['text'],
                        'bbox': item['bbox']
                    }
                    dataset[image_path].append(labels)

                if len(dataset) >= limit:
                    break
            else:
                print(f"Image not found: {image_path}")

    final_dataset = [{'image_path': img_path, 'labels': labels} for img_path, labels in dataset.items()]
    return final_dataset[:limit]

# Define directories
images_dir = '/content/pubtables/subset/img_train'
annotations_dir = '/content/pubtables/subset/words_train'

# Load the dataset with a limit of 100 unique images
train_dataset = load_dataset_with_labels(images_dir, annotations_dir, limit=100)

# Print the first 5 items to check
print(train_dataset[:5])

# Test dataset
print(train_dataset[0:1])

# Subset of 100 examples from the training dataset
subset_training_dataset = train_dataset[:100]

# Check
for img_path in subset_training_dataset:
    print(img_path)


"""Fix the image transformation by ensuring conversion to torch.Tensor during preprocessing."""
def preprocess_function(examples):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    processed_data = []

    for example in examples:
        image_column = 'image_path' if 'image_path' in example else 'image'

        try:
            # Process the image
            pil_image = Image.open(example[image_column]).convert("RGB")
            tensor_image = transform(pil_image)

            # Process the labels
            class_labels = []
            boxes = []

            for item in example['labels']:
                # Assuming each item has a 'bbox' and 'text'
                class_labels.append(0)
                boxes.append(item['bbox'])
            processed_data.append({
                "pixel_values": tensor_image,
                "labels": {
                    "class_labels": torch.tensor(class_labels),
                    "boxes": torch.tensor(boxes)
                }
            })

        except Exception as e:
            print(f"Error processing image: {e}")
            processed_data.append({
                "pixel_values": torch.zeros(3, 224, 224),  # Placeholder
                "labels": {
                    "class_labels": torch.tensor([0]),
                    "boxes": torch.tensor([[0, 0, 0, 0]])
                }
            })

    return processed_data

[{'image_path': '/content/pubtables/subset/img_train/PMC1289288_table_0.jpg', 'labels': [{'text': 'Table', 'bbox': [32.358585858585855, 14.230568181818171, 59.21248737373736, 25.90837121212121]}, {'text': '1:', 'bbox': [61.9843308080808, 14.230568181818171, 70.29986111111111, 25.90837121212121]}, {'text': 'AntiJen', 'bbox': [73.12609848484847, 14.230568181818171, 108.56656565656564, 25.90837121212121]}, {'text': 'sub-databases', 'bbox': [111.39281565656566, 14.230568181818171, 176.95275252525252, 25.90837121212121]}, {'text': 'and', 'bbox': [179.77902777777774, 14.230568181818171, 196.8534217171717, 25.90837121212121]}, {'text': 'content.', 'bbox': [199.67968434343433, 14.230568181818171, 238.9993308080808, 25.90837121212121]}, {'text': 'DATABASE', 'bbox': [37.358585858585855, 36.90358585858584, 97.76209595959597, 48.932601010100996]}, {'text': 'CONTENT', 'bbox': [352.38388888888886, 36.90358585858584, 408.6036616161616, 48.932601010100996]}, {'text': 'T', 'bbox': [37.358585858585855, 

In [12]:
  """When an erroneous sample is encountered during collation, None should be returned instead of batch/{}. The Trainer can then ignore this sample.
  """

  def custom_data_collator(features):
      try:
          # Initialise a dictionary to hold batch data
          batch = {}

          # Stack pixel values into tensor
          batch['pixel_values'] = torch.stack([f['pixel_values'] for f in features])

          # Handle labels if they exist
          if 'labels' in features[0]:
              batch['labels'] = [f['labels'] for f in features]

          return batch

      except Exception as e:
          print(f"Error during collation: {e}")
          return None  # Return None to indicate an erroneous sample


In [13]:
"""Replace the raw dataset with the preprocessed subset."""

# Preprocess the dataset
processed_dataset = preprocess_function(subset_training_dataset)

# Replace the training dataset with the pre-processed subset
train_dataset = [{"pixel_values": item["pixel_values"], "labels": item["labels"]} for item in processed_dataset]

# Print to confirm the structure of train_dataset
print(train_dataset[:5])

[{'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'labels': {'class_labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        

In [14]:
# Convert the processed dataset into a format suitable for the Trainer
train_dataset = [{"pixel_values": item["pixel_values"], "labels": item["labels"]} for item in processed_dataset]

In [15]:
from transformers import TrainingArguments

# Training arguments
training_arguments = TrainingArguments(
    output_dir='./results',                    # Model prediction directory
    evaluation_strategy='no',                     # Eval strategy during training
    save_strategy='epoch',                     # Save strategy during training
    save_total_limit=2,                        # Maximum number of checkpoints to save
    save_steps=100,                            # Save every 100 steps
    learning_rate=2e-5,                        # Learning rate
    per_device_train_batch_size=16,            # Training batch size
    per_device_eval_batch_size=16,             # Evaluation batch size
    num_train_epochs=3,                        # Training epochs
    logging_dir='./logs',                      # Logging directory
    logging_steps=10,                          # Log training
    remove_unused_columns=False,               # Do not remove unused columns in dataset
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
from transformers import Trainer

# Ensure your train_dataset is in the correct format
trainer = Trainer(
    model=model,                               # Trained model
    args=training_arguments,                   # Training arguments
    train_dataset=train_dataset,               # Training dataset (use processed dataset)
    data_collator=custom_data_collator         # Data collator for batch
)


In [17]:
# Define directories for validation data
images_val_dir = '/content/pubtables/subset/img_val'
annotations_val_dir = '/content/pubtables/subset/words_val'

# Load the validation dataset with a limit of 100 unique images (adjust as needed)
val_dataset = load_dataset_with_labels(images_val_dir, annotations_val_dir, limit=100)

# Preprocess the validation dataset
processed_val_dataset = preprocess_function(val_dataset)

# Convert the processed dataset into a format suitable for the Trainer
eval_dataset = [{"pixel_values": item["pixel_values"], "labels": item["labels"]} for item in processed_val_dataset]

# Set the validation dataset in the Trainer
trainer.eval_dataset = eval_dataset

In [18]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,156.929000
20,159.005000


TrainOutput(global_step=21, training_loss=156.17662774948846, metrics={'train_runtime': 172.9409, 'train_samples_per_second': 1.735, 'train_steps_per_second': 0.121, 'total_flos': 7802176899686400.0, 'train_loss': 156.17662774948846, 'epoch': 3.0})

In [19]:
#!pip install transformers pytesseract
#!apt-get install tesseract-ocr

#from transformers import TableTransformerForObjectDetection, AutoImageProcessor, pipeline
#import json
#from PIL import Image
#import pytesseract
#import numpy as np

# Load table detection model
#table_detection = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")
#image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")

# Create a table structure recognition pipeline
table_recogniser = pipeline("object-detection", model="microsoft/table-transformer-structure-recognition")

# Load and preprocess the uploaded table that has been converted to an image
for i in range(len(tables)):
  # Load table image
  test_image = Image.open(f'table_{i}.png').convert("RGB")
  inputs = image_processor(images=test_image, return_tensors="pt")

# Get table structure prediction
predictions = table_recogniser(test_image)

# Convert prediction to JSON
predicted_tables = json.dumps(predictions, indent=5)

# Save the predictions to a JSON file
with open('predicted_tables.json', 'w') as f:
    f.write(predicted_tables)

# Print the predictions
print(predicted_tables)

# Test image
print(test_image)



config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


preprocessor_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

[
     {
          "score": 0.6028777956962585,
          "label": "table row",
          "box": {
               "xmin": 23,
               "ymin": 215,
               "xmax": 478,
               "ymax": 233
          }
     },
     {
          "score": 0.758108377456665,
          "label": "table row",
          "box": {
               "xmin": 25,
               "ymin": 291,
               "xmax": 478,
               "ymax": 312
          }
     },
     {
          "score": 0.8259978890419006,
          "label": "table row",
          "box": {
               "xmin": 23,
               "ymin": 145,
               "xmax": 478,
               "ymax": 163
          }
     },
     {
          "score": 0.5776318907737732,
          "label": "table column",
          "box": {
               "xmin": 210,
               "ymin": 33,
               "xmax": 336,
               "ymax": 336
          }
     },
     {
          "score": 0.7481175661087036,
          "label": "table row",
          

In [20]:
# Process image, convert image to numpy array
image_np = np.array(test_image)

# Extract text from Predictions using pytesseract OCR
extracted_data = []
for predict in predictions:
    if predict['score'] > 0.5:  # Filter predictions based on confidence score
        box = predict['box']    # Extract bounding box
        x0, y0, x1, y1 = map(int, [box['xmin'], box['ymin'], box['xmax'], box['ymax']])  # Convert bounding box to integer and store in integer coordinates
        cell_image = image_np[y0:y1, x0:x1]  # Crop cell area using boundary coordinates
        text = pytesseract.image_to_string(cell_image)  # Extract text from cell_image
        extracted_data.append(text.strip())  # Store text extracted

extracted_data = [text for text in extracted_data if text and not text.isspace()] # ensures no extra white space and quotes
# Combine prediction and table content
join_data = {
    "predicted_tables": json.loads(predicted_tables),
    "extracted_table_data": extracted_data
}

# Save prediction and extracted table data to a JSON file
with open('join_data.json', 'w') as f:
    json.dump(join_data, f, indent=5)

# Print combined data
print(json.dumps(join_data, indent=5))

# Print extracted table data
table_data_extracted = {
    "extracted_table_data": extracted_data
}

# Save extracted table data to a JSON file
with open('table_data_extracted.json', 'w') as f:
    json.dump(table_data_extracted, f, indent=5)

# Print extracted table data
print(json.dumps(table_data_extracted, indent=5))

{
     "predicted_tables": [
          {
               "score": 0.6028777956962585,
               "label": "table row",
               "box": {
                    "xmin": 23,
                    "ymin": 215,
                    "xmax": 478,
                    "ymax": 233
               }
          },
          {
               "score": 0.758108377456665,
               "label": "table row",
               "box": {
                    "xmin": 25,
                    "ymin": 291,
                    "xmax": 478,
                    "ymax": 312
               }
          },
          {
               "score": 0.8259978890419006,
               "label": "table row",
               "box": {
                    "xmin": 23,
                    "ymin": 145,
                    "xmax": 478,
                    "ymax": 163
               }
          },
          {
               "score": 0.5776318907737732,
               "label": "table column",
               "box": {
                    "